# Variational Auto Encoder (VAE)

In [12]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Reshape, Input, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt

# Load MNIST dataset
(x_train, _), (x_test, _) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

In [15]:
import matplotlib.pyplot as plt

In [17]:
x_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [22]:
# Network parameters
input_shape = (x_train.shape[1], )
intermediate_dim = 64
latent_dim = 2

# VAE model = encoder + decoder
# build encoder model
inputs = Input(shape=input_shape, name='encoder_input')
x = Dense(intermediate_dim, activation='relu')(inputs)
z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)


In [23]:
# Use reparameterization trick to push the sampling out as input
z = Lambda(lambda a: a[0] + tf.exp(a[1] / 2) * tf.random.normal(tf.shape(a[0])), 
           output_shape=(latent_dim,))([z_mean, z_log_var])

In [24]:
# instantiate encoder model
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 encoder_input (InputLayer)  [(None, 784)]                0         []                            
                                                                                                  
 dense_3 (Dense)             (None, 64)                   50240     ['encoder_input[0][0]']       
                                                                                                  
 z_mean (Dense)              (None, 2)                    130       ['dense_3[0][0]']             
                                                                                                  
 z_log_var (Dense)           (None, 2)                    130       ['dense_3[0][0]']             
                                                                                            

In [25]:
# build decoder model
latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
x = Dense(intermediate_dim, activation='relu')(latent_inputs)
outputs = Dense(input_shape[0], activation='sigmoid')(x)

In [26]:
# instantiate decoder model
decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()


Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 z_sampling (InputLayer)     [(None, 2)]               0         
                                                                 
 dense_4 (Dense)             (None, 64)                192       
                                                                 
 dense_5 (Dense)             (None, 784)               50960     
                                                                 
Total params: 51152 (199.81 KB)
Trainable params: 51152 (199.81 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
# instantiate VAE model
outputs = decoder(encoder(inputs)[2])
vae = Model(inputs, outputs, name='vae_mlp')

In [28]:
vae.summary()

Model: "vae_mlp"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 784)]             0         
                                                                 
 encoder (Functional)        [(None, 2),               50500     
                              (None, 2),                         
                              (None, 2)]                         
                                                                 
 decoder (Functional)        (None, 784)               51152     
                                                                 
Total params: 101652 (397.08 KB)
Trainable params: 101652 (397.08 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
# VAE loss = mse_loss or xent_loss + kl_loss
reconstruction_loss = tf.keras.losses.binary_crossentropy(inputs, outputs)
reconstruction_loss *= input_shape[0]
kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
kl_loss = tf.reduce_sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = tf.reduce_mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)

vae.compile(optimizer='adam')
vae.summary()

Model: "vae_mlp"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 encoder_input (InputLayer)  [(None, 784)]                0         []                            
                                                                                                  
 encoder (Functional)        [(None, 2),                  50500     ['encoder_input[0][0]']       
                              (None, 2),                                                          
                              (None, 2)]                                                          
                                                                                                  
 decoder (Functional)        (None, 784)                  51152     ['encoder[0][2]']             
                                                                                            

In [30]:
# Train the autoencoder
vae.fit(x_train, epochs=50, batch_size=256, validation_data=(x_test, None))

Epoch 1/50
235/235 [==============================] - 3s 8ms/step - loss: 253.4352 - val_loss: 197.3993
Epoch 2/50
235/235 [==============================] - 1s 6ms/step - loss: 191.5365 - val_loss: 187.1571
Epoch 3/50
235/235 [==============================] - 1s 6ms/step - loss: 185.3945 - val_loss: 182.4924
Epoch 4/50
235/235 [==============================] - 2s 7ms/step - loss: 181.0128 - val_loss: 177.9671
Epoch 5/50
235/235 [==============================] - 2s 7ms/step - loss: 176.8332 - val_loss: 174.4063
Epoch 6/50
235/235 [==============================] - 2s 7ms/step - loss: 173.8447 - val_loss: 172.1302
Epoch 7/50
235/235 [==============================] - 2s 7ms/step - loss: 171.8895 - val_loss: 170.7113
Epoch 8/50
235/235 [==============================] - 2s 8ms/step - loss: 170.5973 - val_loss: 169.5439
Epoch 9/50
235/235 [==============================] - 2s 8ms/step - loss: 169.4253 - val_loss: 168.5776
Epoch 10/50
235/235 [==============================] - 2s 8ms/st

In [31]:
# Display a 2D plot of the digit classes in the latent space
def plot_latent_space(vae, n=30, figsize=15):
    # display a n*n 2D manifold of digits
    digit_size = 28
    figure = np.zeros((digit_size * n, digit_size * n))
    # linearly spaced coordinates corresponding to the 2D plot
    # of digit classes in the latent space
    grid_x = np.linspace(-4, 4, n)
    grid_y = np.linspace(-4, 4, n)

    for i, yi in enumerate(grid_y):
        for j, xi in enumerate(grid_x):
            z_sample = np.array([[xi, yi]])
            x_decoded = decoder.predict(z_sample)
            digit = x_decoded[0].reshape(digit_size, digit_size)
            figure[i * digit_size: (i + 1) * digit_size,
                   j * digit_size: (j + 1) * digit_size] = digit

    plt.figure(figsize=(figsize, figsize))
    plt.imshow(figure, cmap='Greys_r')
    plt.show()

plot_latent_space(vae, n=50)


1/1 [==============================] - 0s 23ms/step


In [ ]:
def generate_digit_image(model, latent_point):
    """ Generate a digit image from a point in the latent space """
    # Predict the output (image) for the given latent point
    generated_image = model.predict(np.array([latent_point]))
    digit_image = generated_image[0].reshape(28, 28)

    # Plot the image
    plt.imshow(digit_image, cmap='gray')
    plt.show()

# Example usage:
generate_digit_image(decoder, [3, 3])  # Replace [0, 1] with any coordinates in latent space
